# Import wall properties directly from IFC file

# 1. Preliminaries: installs and imports

In [1]:
from shapely.geometry import LineString, Polygon

Install ifcopenshell

In [2]:
# Install ifcopenshell package
!pip install ifcopenshell

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 7.5 MB/s eta 0:00:00


Import necessary packages

In [5]:
import os
import ifcopenshell
from ifcopenshell.geom import create_shape
import pandas as pd
import numpy as np

Mount drive so that nonstandard python files can be included and IFC file can be imported.
You'll be prompted to authenticate your Google account.
After mounting, your Drive is available at
/content/drive/My Drive/

In [6]:
# Mount google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Here you need to add to the path the folder that I shared with you containing the necessary python files (in my case it is under projects/BUILDCHAIN/IFC_files/BIM_Pilot4). You can put that to a chosen path by the following steps:


1.   Go to Google Drive
2.   Locate the shared file (check "Shared with me")
3.   Right-click the file → Click "Add shortcut to Drive" (if it's shared)
4.   Navigate to your chosen folder
5.   Click "Move here" or "Add shortcut here"


In [7]:

import sys
sys.path.append("/content/drive/MyDrive/projects/BUILDCHAIN/IFC_to_wall_properties")
from building import Building
from utils import plot_by_plotly

In [8]:
from IPython.display import display

# 2. Check available IFC files, choose one for further testing

Define path of folder where ifc files are located

In [9]:
ifc_folder_path = "/content/drive/My Drive/projects/BUILDCHAIN/IFC_files/BIM_Pilot4"

In [10]:
# List files
for filename in os.listdir(ifc_folder_path):
    print(filename)

BUILDCHAIN_Pilot4_Carducci.ifc
BUILDCHAIN_Pilot4_Locchi.ifc
BUILDCHAIN_Pilot4_A_Del Sarto.ifc
BUILDCHAIN_Pilot4_Agnesi.ifc
BUILDCHAIN_Pilot4_Garibaldi.ifc
Test4walls.ifc
Test4walls_2floors.ifc
Progetto1N-2floors.ifc
Progetto1N-1floor.ifc
Progetto1N-1floor-vertic_load.ifc
Project1N-1floor-vertic_loads_floors_loadbwalls.ifc
Progetto1N-1floor-CompressiveStrength-G1-G2-QK1-LoadDir-LoadDistrib.ifc
Progetto1N-1floor-CompressiveStrength.ifc
Benchmark_CaseA_mechprop.ifc
Macchiavelli_Qkcat_modifiedfloors.ifc
Macchiavelli_Qkcat_modifiedfloors_mod_fu.ifc


In [9]:
ifc_file_path = ifc_folder_path + "/Macchiavelli_Qkcat_modifiedfloors_mod_fu.ifc"

In [10]:
ifc_file = ifcopenshell.open(ifc_file_path)

# 3. Create building object collecting all wall and slab ifc elements

## 3.1 Create building object

In [11]:
# Create B Building object - use the commented out command if
#B = IfcObjectSet(ifc_file, only_geometry=True)
B = Building(ifc_file, only_geometry=False)

Distance of bounding box point [1. 1. 5.] and closest element point is 0.10499999999999887>0.1, meaning the element 9827 is not rectangular, which can cause inaccuracies 
Distance of bounding box point [1. 1. 6.] and closest element point is 0.10499999999999887>0.1, meaning the element 9827 is not rectangular, which can cause inaccuracies 
Distance of bounding box point [73. 42.  5.] and closest element point is 0.471637073520327>0.1, meaning the element 10639 is not rectangular, which can cause inaccuracies 
Distance of bounding box point [73. 42.  6.] and closest element point is 0.471637073520327>0.1, meaning the element 10639 is not rectangular, which can cause inaccuracies 
Distance of bounding box point [73. 42. 11.] and closest element point is 0.471637073520327>0.1, meaning the element 11604 is not rectangular, which can cause inaccuracies 
Distance of bounding box point [73. 42. 11.] and closest element point is 0.471637073520327>0.1, meaning the element 11604 is not rectangul

In [12]:
W = B.find_object_by_id(4572)
W

mesh =B.get_all_meshes_highlighting_elements_with_id([W.id, 622, 1928, 3003])
plot_by_plotly(mesh, title="Selected Wall", showlegend=True)

## 3.2 Assemble table of wall partitions properties for push over analysis

In [13]:
# Assemble wall partition properties
props = B.get_wall_partition_properties(detailed=False)
# Set precision of displayed numbers in the table
pd.set_option('display.precision', 2)
# Show wall partition properties for floor 0
props[0]

Wall part above window of wall 6795 can not be supported by the end of wall part


,L [m],w [m],H [m],Cx [m],Cy [m],α,σ [N/mm²],τ [N/mm²],fₘ [N/mm²],γ [kN/m³],E [N/mm²],G [N/mm²]
Wall,,,,,,,,,,,,
1,1.25,0.96,5.8,0.84,0.70,0.0,0.19,0.05,2.33,19.62,619.2,103.2
2,4.97,0.96,5.8,6.45,0.70,0.0,0.19,0.05,2.33,19.62,619.2,103.2
3,1.00,0.96,5.8,11.94,0.70,0.0,0.19,0.05,2.33,19.62,619.2,103.2
4,2.00,0.56,5.8,13.44,0.70,0.0,0.22,0.05,2.33,19.62,619.2,103.2
5,2.35,0.56,5.8,17.41,0.70,0.0,0.22,0.05,2.33,19.62,619.2,103.2
...,...,...,...,...,...,...,...,...,...,...,...,...
157,4.18,0.26,5.8,33.64,7.06,90.0,0.29,0.05,2.33,19.62,619.2,103.2
158,2.68,0.26,5.8,33.64,10.49,90.0,0.29,0.05,2.33,19.62,619.2,103.2
159,1.36,0.36,5.8,16.40,46.22,90.0,0.22,0.05,2.33,19.62,619.2,103.2


In [14]:
# Show properties of wall partitions of floor 1
props[1]

,L [m],w [m],H [m],Cx [m],Cy [m],α,σ [N/mm²],τ [N/mm²],fₘ [N/mm²],γ [kN/m³],E [N/mm²],G [N/mm²]
Wall,,,,,,,,,,,,
110,1.55,0.56,5.35,72.39,41.92,86.57,0.10,0.05,2.33,19.62,619.2,103.2
111,2.28,0.56,5.35,72.61,45.63,86.57,0.10,0.05,2.33,19.62,619.2,103.2
112,2.40,0.56,5.35,72.86,49.76,86.57,0.10,0.05,2.33,19.62,619.2,103.2
64,1.63,0.56,5.35,73.07,53.31,86.57,0.10,0.05,2.33,19.62,619.2,103.2
108,1.06,0.48,5.35,67.75,50.59,90.00,0.09,0.05,2.33,19.62,619.2,103.2
...,...,...,...,...,...,...,...,...,...,...,...,...
115,1.95,0.48,5.35,67.75,48.60,90.00,0.11,0.05,2.33,19.62,619.2,103.2
160,1.34,0.36,5.35,16.40,48.91,90.00,0.10,0.05,2.33,19.62,619.2,103.2
138,4.67,0.48,5.35,8.10,12.11,0.00,0.10,0.05,2.33,19.62,619.2,103.2


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# The old version of the table with more details

In [16]:
# Assemble wall partition properties
props = B.get_wall_partition_properties(detailed=True)
# Set precision of displayed numbers in the table
pd.set_option('display.precision', 2)
# Show wall partition properties for floor 0
props[0]

Wall part above window of wall 6795 can not be supported by the end of wall part


,WallIfcId,PartitionNumber,L [m],w [m],H [m],Cx [m],Cy [m],α,γ [kN/m³],σ {SLU} [N/mm²],σ [N/mm²],E [N/mm²],G [N/mm²],f_u [N/mm²],τ [N/mm²],fₘ [N/mm²],nu
Wall,,,,,,,,,,,,,,,,,
1,106,106_1,1.25,0.96,5.8,0.84,0.70,0.0,19.62,0.22,0.19,619.2,103.2,0.08,0.05,2.33,0.2
2,106,106_2,4.97,0.96,5.8,6.45,0.70,0.0,19.62,0.22,0.19,619.2,103.2,0.08,0.05,2.33,0.2
3,106,106_3,1.00,0.96,5.8,11.94,0.70,0.0,19.62,0.22,0.19,619.2,103.2,0.08,0.05,2.33,0.2
4,326,326_1,2.00,0.56,5.8,13.44,0.70,0.0,19.62,0.28,0.22,619.2,103.2,0.08,0.05,2.33,0.2
5,326,326_2,2.35,0.56,5.8,17.41,0.70,0.0,19.62,0.28,0.22,619.2,103.2,0.08,0.05,2.33,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,7930,7930_1,4.18,0.26,5.8,33.64,7.06,90.0,19.62,0.37,0.29,619.2,103.2,0.08,0.05,2.33,0.2
158,7987,7987_1,2.68,0.26,5.8,33.64,10.49,90.0,19.62,0.37,0.29,619.2,103.2,0.08,0.05,2.33,0.2
159,8030,8030_1,1.36,0.36,5.8,16.40,46.22,90.0,19.62,0.24,0.22,619.2,103.2,0.08,0.05,2.33,0.2


In [17]:
np.unique(props[0]["f_u [N/mm²]"])

array([0.0774])

In [18]:
np.unique(props[1]["f_u [N/mm²]"])

array([0.0774])

# Get floor properties (floor heights and weights)

In [19]:
df = B.get_floor_properties()
df

,H [m],W [kN]
Floor1,5.8,31455.56
Floor2,5.3,17936.79


### Get floor heights

In [20]:
pd.DataFrame(B.floor_heights)

,0,1
BottomHeight,0.0,5.8
TopHeight,5.8,11.1


## 3.3 Vizualize building, building elements

Show all slabs and walls

In [21]:
# Create mesh of all building elements
B_mesh = B.get_all_meshes(opacity=0.9)
# Plot mesh
plot_by_plotly(B_mesh, title="Building", showlegend=True)

In [ ]:
mesh = W.get_mesh()
points_scatter = W.get_plotly_scatterpoints()
#wpmesh= W.wall_partitions.objects[0].get_mesh()

plot_by_plotly([mesh, points_scatter], title="Wall", showlegend=True)


In [ ]:
W_side = B.find_object_by_id(326)
mesh_side = W_side.get_mesh(color="green")
points_scatter = W_side.get_plotly_scatterpoints()
W_top = B.find_object_by_id(4064)
top_mesh = W_top.get_mesh(color="red")
plot_by_plotly([mesh, points_scatter, top_mesh, mesh_side], title="Wall", showlegend=True)

Show all elements, but highlight a list of specific elements with given ids

In [ ]:
# Create grey mesh of all elements and highlighted (orange)
# ones of elements
B_highlighted_mesh = B.get_all_meshes_highlighting_elements_with_id([6795, 3341])
# vizalize the created mesh
plot_by_plotly(B_highlighted_mesh, title="Building", showlegend=True)

Show only the slabs

In [ ]:
# Create mesh of all slabs
B_mesh_slabs = B.get_mesh_by_type("IfcSlab", opacity = 1)
W_mesh = W.get_mesh(color="orange")
# Plot the created mesh
plot_by_plotly(B_mesh_slabs, title="Slabs", showlegend=True)

Look for one specific element based on ID number

In [ ]:
W2 = B.find_object_by_id(5090)
W2
W3 = B.find_object_by_id(2426) # lower end wall
W4 = B.find_object_by_id(4064)  #upper end wall
W2, W3, W4

(<Wall #5090>, <Wall #2426>, <Wall #4064>)

In [ ]:
W3.wall_partitions.objects

[<SolidObject with 8 points and 12 faces>,
 <SolidObject with 8 points and 12 faces>]

In [ ]:
W4.supported_by_walls

[{'Object': <Wall #326>,
  'ContactLength': np.float64(0.56),
  'ContactStart': array([38.09470893,  0.41999985,  5.8       ]),
  'ContactEnd': array([38.09470893,  0.97999985,  5.8       ])},
 {'Object': <Wall #738>,
  'ContactLength': np.float64(0.5600000000000005),
  'ContactStart': array([38.09470893, 11.8296493 ,  5.8       ]),
  'ContactEnd': array([38.09470893, 12.3896493 ,  5.8       ])},
 {'Object': <Wall #2426>,
  'ContactLength': np.float64(10.849649449999998),
  'ContactStart': array([38.09470893,  0.97999985,  5.8       ]),
  'ContactEnd': array([38.09470893, 11.8296493 ,  5.8       ])}]

In [ ]:
W3.wall_partitions.objects

[<SolidObject with 8 points and 12 faces>,
 <SolidObject with 8 points and 12 faces>]

In [ ]:
filtered_objects = B.filter_objects_by_floor(1)
mesh = B.get_all_meshes_highlighting_elements_with_id([5090, 11676, 11898],
                                                      objects=filtered_objects)
plot_by_plotly(mesh, title="Selected Wall", showlegend=True)

In [ ]:
W2.wall_partitions.objects

[<SolidObject with 8 points and 12 faces>,
 <SolidObject with 8 points and 12 faces>,
 <SolidObject with 8 points and 12 faces>,
 <SolidObject with 8 points and 12 faces>,
 <SolidObject with 8 points and 12 faces>,
 <SolidObject with 8 points and 12 faces>,
 <SolidObject with 8 points and 12 faces>,
 <SolidObject with 8 points and 12 faces>,
 <SolidObject with 8 points and 12 faces>,
 <SolidObject with 8 points and 12 faces>,
 <SolidObject with 8 points and 12 faces>,
 <SolidObject with 8 points and 12 faces>,
 <SolidObject with 8 points and 12 faces>]

In [ ]:
W2.supported_by_walls

[{'Object': <Wall #1394>,
  'ContactLength': np.float64(60.39092983133414),
  'ContactStart': array([12.43502885, 53.84812606,  5.8       ]),
  'ContactEnd': array([72.82595868, 53.84812606,  5.8       ])}]

In [ ]:
mesh = W2.get_mesh()
points_scatter = W2.get_plotly_scatterpoints()
plot_by_plotly([mesh, points_scatter, ], title="Wall", showlegend=True)

In [ ]:
# Check which slab partitions are loading this wall
[link_i["Object"].id for link_i in W2.loaded_by_slab_partition]

['11676_2', '11898_2']

In [ ]:
# Look for Wall #1394
W = B.find_object_by_id(1394)
# Display wall element
W

<Wall #1394>

Check which wall and slab-part is loading this specific wall, and which is supporting it

In [ ]:
# Check contacts with upper walls
 # (the resulted "contact_with_upper_walls" is a
 # list of dictionaries with keys "Object" - storing the wall object #1428, which is on top of this wall -
 # and "ContactLength" - storing the length of contact)
contact_with_upper_walls = W.loaded_by_walls
contact_with_upper_walls

[{'Object': <Wall #3864>,
  'ContactLength': np.float64(0.5610046912899699),
  'ContactStart': array([73.0896811 , 53.56812606,  5.8       ]),
  'ContactEnd': array([73.12324095, 54.12812606,  5.8       ])},
 {'Object': <Wall #3972>,
  'ContactLength': np.float64(7.105427357601002e-15),
  'ContactStart': array([67.74671804, 53.56812606,  5.8       ]),
  'ContactEnd': array([67.74671804, 53.56812606,  5.8       ])},
 {'Object': <Wall #5090>,
  'ContactLength': np.float64(60.39092983133414),
  'ContactStart': array([12.43502885, 53.84812606,  5.8       ]),
  'ContactEnd': array([72.82595868, 53.84812606,  5.8       ])}]

In [ ]:
# Check contacts with loading slabs
 # (the resulted "contact_with_slabs" is a
 # list of dictionaries with keys "Object" - storing the wall object -
 # and "ContactLength" - storing the length of contact with this upper wall)
contact_with_slabs = W.loaded_by_slab_partition
contact_with_slabs

[{'Object': <SolidObject with 8 points and 12 faces>,
  'ContactLength': np.float64(4.852900519999963)}]

In [ ]:
# Check contacts with underneath walls
contact_with_supporting_walls = W.supported_by_walls
contact_with_supporting_walls

[]

In [ ]:
W.floor

0

Plot the selected wall and all these contacting elements, elements-part

In [ ]:
# Create mesh of selected wall
W_mesh = [W.get_mesh(color="orange")]
# Create mesh of loading walls
mesh_loading_walls = [contact_i["Object"].get_mesh(
    color="grey", opacity=0.8) for contact_i in contact_with_upper_walls]
# Create mesh of loading slabs
mesh_loading_slabs = [contact_i["Object"].get_mesh(
    color="green", opacity=0.8) for contact_i in contact_with_slabs]
# Create mesh of supporting walls
mesh_supporting_walls = [contact_i["Object"].get_mesh(
    color="grey", opacity=0.8) for contact_i in contact_with_supporting_walls]
# Merge meshes
meshes = mesh_loading_walls + mesh_loading_slabs + mesh_supporting_walls + W_mesh
# Plot all these elements
plot_by_plotly(meshes, title="Loading Walls", showlegend=True)

Plot wall parts for push over analysis with wall parts loading it

In [ ]:
# Generate meshes for plotting wall partitions with loading elements (from above opening parts)
wall_partition_mesh = B.get_mesh_of_wall_partitions()
# Plot generated mesh
plot_by_plotly(wall_partition_mesh, title="Wall Partitions and linked wall part loading the partitions", showlegend=True)

Wall part above window of wall 6795 can not be supported by the end of wall part


Plot only wall parts for push over analysis without loading wall parts

In [ ]:
# Generate meshes without loading elements
wall_partition_mesh_no_loading = B.get_mesh_of_wall_partitions(mode="only_main")
# Plot generated meshes
plot_by_plotly(wall_partition_mesh_no_loading, title="Wall Partitions", showlegend=True)

Wall part above window of wall 6795 can not be supported by the end of wall part


## 3.4 Show linkages between elements

### For one specific slab element

Choose one slab element and plot it

In [ ]:
# Check all slabs and choose one to present functionality
slabs = B.filter_objects_by_type("IfcSlab")
# Select one
S = slabs[0]
# Generate mesh by highlighting this slab element
mesh_with_S = B.get_all_meshes_highlighting_elements_with_id(S.id)
# Plot mesh
plot_by_plotly(mesh_with_S, title="Slab", showlegend=True)

In [ ]:
# Check the slab parts of slabs that are transmitting loads onto one wall
SPs = S.slab_partitions.objects
SPs

[<SolidObject with 8 points and 12 faces>,
 <SolidObject with 8 points and 12 faces>]

Check to which walls the slab parts transmit their loads

In [ ]:
# Check on which wall the first slab partition is loading
Ws_SP1 = SPs[0].supporting_elements
# Check on which wall the second slab partition is loading
Ws_SP2 = SPs[1].supporting_elements
# Show loading walls
Ws_SP1, Ws_SP2

([<Wall #7930>, <Wall #7987>], [<Wall #2426>])

 Plot the slab parts and the walls they load onto

In [ ]:
# Mesh of the first slab partition
SP_1_mesh = SPs[0].get_mesh(color="orange")
# Mesh of the second slab partition
SP_2_mesh = SPs[1].get_mesh(color="grey")
# Mesh of the supporting walls
W_1_mesh = Ws_SP1[0].get_mesh(color="orange")
W_2_mesh = Ws_SP1[1].get_mesh(color="orange")

W_3_mesh = Ws_SP2[0].get_mesh(color="grey")
# Plot all these meshes
plot_by_plotly([SP_1_mesh, SP_2_mesh, W_1_mesh, W_2_mesh, W_3_mesh], title="Slab Partitions", showlegend=True)

In [ ]:
len(Ws_SP2[0].loaded_by_slab_partition)

1

In [ ]:

objects = B.filter_objects_by_floor(0)
mesh = B.get_all_meshes_highlighting_elements_with_id(2426, objects=objects)
plot_by_plotly(mesh, title="Selected Wall", showlegend=True)

### For one specific wall element

Select one wall element

In [ ]:
# Find wall objects
walls = B.filter_objects_by_type("IfcWall")
# Select one
W = walls[9]
W

Vizualize it

In [ ]:
# Generate mesh with highlighted element
B_mesh_W =B.get_all_meshes_highlighting_elements_with_id(W.id)
plot_by_plotly(B_mesh_W, title= "Selected Wall Element", showlegend=True)

Check which slab part load onto this wall

In [ ]:
# Check contact info
contact = W.loaded_by_slab_partition
contact

In [ ]:
# The slab part loading onto the wall:
SP1 = contact[0]["Object"]
SP2 = contact[1]["Object"]
# Show slab part and its id (2054 is the Id of the slab)
SP1, SP1.id, SP2, SP2.id

Check which wall load onto this wall

In [ ]:
# Check contact info
contact = W.loaded_by_walls
# Check wall id
W_top = contact[0]["Object"]
W_top

Check onto which wall this wall transmits its forces

In [ ]:
# Check contact info
contact = W.supported_by_walls
contact

Plot all these elements

In [ ]:
mesh_W = W.get_mesh(color="orange")
mesh_SP1 = SP1.get_mesh(color="red")
mesh_SP2 = SP2.get_mesh(color="red")
mesh_W_top = W_top.get_mesh(color="red")
#mesh_W_bottom = W_bottom.get_mesh(color="grey")
plot_by_plotly([mesh_W, mesh_SP1, mesh_SP2, mesh_W_top], title="Wall", showlegend=True)

In [ ]:
mesh = B.get_all_meshes_highlighting_elements_with_id([5090, 1394, 12932, 13006], objects=objects)
plot_by_plotly(mesh, title="Selected Wall", showlegend=True)

### Show all links with tables

Show for each wall which slabs and which slab parts load it

In [ ]:
B.get_wall_slab_links()

Or the other way around: show for each slab part on which wall they transmit their loads

In [ ]:
B.get_slab_wall_links()

Show which wall is loading onto which wall

In [ ]:
B.get_wall_below_wall_links()

Show linking wall parts

In [ ]:
B.get_WP_WP_links()

## 3.5 Filtering by floors

In [ ]:
# Check which floors are located on the different floors
B.floor_wall_dict

In [ ]:
# Check all walls that are on fllor 0
B.filter_walls_by_floor(0)

In [ ]:
mesh_ground_floor = B.get_mesh_by_floor(0, opacity=0.8)
plot_by_plotly(mesh_ground_floor, title="Ground floor", showlegend=True)

## 3.6 Filter elements by type

In [ ]:
walls = B.filter_objects_by_type("IfcWall")
slabs = B.filter_objects_by_type("IfcSlab")

In [ ]:
slabs, walls

## 3.7 Show element properties

Show properties of a slab

In [ ]:
# Select one slab
S = B.filter_objects_by_type("IfcSlab")[0]
S

In [ ]:
# Show IFC properties that are imported directly, and used for the computation
S.properties.show()

In [ ]:
# Access these values
S.properties.g1, S.properties.g2, S.properties.q_A, S.properties.load_direction, S.properties.load_distribution

In [ ]:
# Show all general and material properties of the IFC object
S.properties.show_ifc_general_and_material_properties()

Check whether a certain property is given in the ifc object

In [ ]:
is_declared, prop_type, prop = S.properties.is_property_declared("LoadBearing")
is_declared, prop_type, prop

Show properties of a wall

In [ ]:
# Select one wall object
W = B.filter_objects_by_type("IfcWall")[0]
W

In [ ]:
# Show properties directly imported as an attribute to the wall.properties class
W.properties.show()

In [ ]:
# Access these properties
W.properties.density, W.properties.E, W.properties.G,  W.properties.f_u, W.properties.f_c, W.properties.nu, W.properties.g1, W.properties.g2, W.properties.q_A

In [ ]:
# Print out all general and material properties of the ifc object
W.properties.show_ifc_general_and_material_properties()

## 3.8 Show geometrical properties of walls and slabs

Show dimensions and position of wall

In [ ]:
W.get_dimensions()

In [ ]:
S.get_dimensions()

Check global and local coordinates of points of the wall or slab, and its local coordinate system

In [ ]:
# Global coordinates of the element
S.points

In [ ]:
# Local coordinates of the element
S.transform_to_local(S.points)

In [ ]:
# Get angle between local and global x axis
S.get_angle_of_axis()

In [ ]:
# Get origin point of local coordinate system and vectors of local x, y, z
S.get_origin_and_local_axes()

Plot points, element and local coordinate system

In [ ]:
# Get mesh of the slab
mesh = S.get_mesh(name="Slab")
# Get mesh of the slab partitions that load onto one wall
mesh_parts = S.slab_partitions.get_meshes()
# Get plotly scatter object for plotting the points of the slab
point_scatter = S.get_plotly_scatterpoints(name="Points of slab")
# Get cone object for showing the local coordinate system
local_coord = S.get_plotly_local_coord()
# Plot all
data = [mesh, point_scatter, local_coord]+mesh_parts
plot_by_plotly(data, title="Slab", showlegend=True)

Check surface area of slab partitions

In [ ]:
# Surface areas of slab partitions
[P_i.A for P_i in S.slab_partitions.objects]

In [ ]:
# Points of edge of slab partition where the loads are transmitted to the wall
[P_i.e_points for P_i in S.slab_partitions.objects]

In [ ]:
# Check other properties of the slab partition
S.slab_partitions.objects[0].__dict__

## 3.9 Mechanical properties

Show local direction of load transfering, and load distribution property

In [ ]:
S.load_principal_direction, S.load_transfer_type

In [ ]:
S.properties

Show distributed structural pernament load (g1), non-structural pernament load (g2) and variable load (q) transfered via the edge of the slab:

In [ ]:
# Get first part of the slab
SP = S.slab_partitions.objects[0]
# Show loadings acting on the wall from the slab loads
SP.p_g1, SP.p_g2, SP.p_q

Show loads on wall top from slab

In [ ]:
g1, g2, q = W.get_distributed_loads_from_slabs()
g1, g2, q

Show load on wall top from above wall

In [ ]:
g1, g2, q = W.get_distributed_loads_from_above_walls()
g1, g2, q

Total distributed loads in the bottom of the wall

In [ ]:
W.p_g1, W.p_g2, W.p_q

Show stresses in the bottom of the wall

In [ ]:
W.get_limit_state_stress(limit_state="SLE")

In [ ]:
W.get_limit_state_stress(limit_state="SLU")